#### Day 15 - A
Complete the robot instructions and monitor the warehouse layout

In [189]:
#Import Libraries and settings
from copy import deepcopy

settings = {
    "day": 15,
    "test_data": 0
}

In [190]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    #Save grid to 2D array and inputs to seperate array
    at_grid = True
    inputs = []
    grid = []
    for idx_y, line in enumerate(lines):

        #Check if the grid has finished
        if line == "":
            at_grid = False
        else:
            if at_grid:
                if "@" in line:
                    robot_loc = (line.find("@"), idx_y)
                grid.append(list(line))
            else:
                inputs += list(line)

    return grid, inputs, robot_loc

grid_base, inputs, starting_loc = load_input(settings)

In [191]:
grid = deepcopy(grid_base)

In [192]:
def check_loc(grid, loc):
    return grid[loc[1]][loc[0]]

In [193]:
#Convert compass direction into a dir tuple
def translate_step(dir_g, mag=1):
    v = 0
    h = 0

    if "^" in dir_g:
        v = -mag
    elif "v" in dir_g:
        v = mag

    if ">" in dir_g:
        h = mag
    elif "<" in dir_g:
        h = -mag

    return (h, v)

#Apply a direction to a location
def apply_move(loc, dir, mag=1):
    step = translate_step(dir, mag)
    return (loc[0]+step[0], loc[1]+step[1])

#Test the next move
def try_next_move(grid, c_loc, dir):
    #Get character for current location
    c_char = check_loc(grid, c_loc)
    #Get the next location and character currently in the space
    next_loc = apply_move(c_loc, dir)
    next_char = check_loc(grid, next_loc)

    #If wall, do not return a move
    if (next_char == "#"):
        return []
    #If open space, return a move
    elif (next_char == "."):
        return [(c_char, c_loc, next_loc)]
    #If a box, check if the box can move and domino effect
    elif (next_char == "O"):
        #Get the fallout from moving this box
        res = try_next_move(grid, next_loc, dir)
        #If no res then box cannot move
        if res == []:
            return []
        #If res then move robot along with box(es)
        else:
            return [(c_char, c_loc, next_loc)] + res

#Apply a list of moves to the grid
def apply_next_moves(grid, moves):
    
    #Apply moves in reverse
    for move in moves[::-1]:
        old_loc = move[1]
        new_loc = move[2]

        #Check if the character moving is the robot
        if move[0] == "@":
            #Set n_loc so the robot's position can be updated
            n_loc = new_loc

        grid[new_loc[1]][new_loc[0]] = grid[old_loc[1]][old_loc[0]]
        grid[old_loc[1]][old_loc[0]] = "."

    return n_loc

In [194]:
#Apply the input to the robot
def apply_instructions(grid, inputs, c_loc):
    for dir in inputs:
        moves = try_next_move(grid, c_loc, dir)
        if moves:
            c_loc = apply_next_moves(grid, moves)

def calculate_score(grid):
    subtotal = 0
    for idx_y, line in enumerate(grid):
        for idx_x, space in enumerate(line):
            if space == "O":
                subtotal += (100*idx_y) + idx_x

    return subtotal

In [195]:
apply_instructions(grid, inputs, starting_loc)

In [196]:
calculate_score(grid)

1517819